<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.2: Generators: Collections
**Prev: [Generators: Parameters](3.1_parameters.ipynb)**<br>
**Next: [Interlude: Chisel Standard Library](3.2_interlude.ipynb)**


## Motivation
제너레이터는 IO, Module, 테스트 벡터 등 다양한 수의 객체를 처리해야 하는 경우가 많습니다.
컬렉션은 이와 같은 상황을 처리하기 위한 중요한 빌딩 블록입니다.
이 모듈은 Scala 컬렉션과 Chisel 생성기와 함께 사용하는 방법을 소개합니다.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/parkdongho/dev/chisel-bootcamp-kr/source/load-ivy.sc"

`mutable.ArrayBuffer`가 `scala.collections`에 있기 때문에 여기에 새 가져오기를 추가합니다.

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import scala.collection._

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import scala.collection._

---
# Generators and Collections<a name="generators-and-collections"></a> 
이 섹션에서는 *제너레이터* 의 개념과 이를 구현하기 위한 도구로 Scala 컬렉션의 사용에 초점을 맞출 것입니다.
Chisel 코드를 회로의 *인스턴스*, 즉 특정 회로에 대한 설명으로 보는 대신,
대신 회로의 생성기로 간주합니다.

이전 연습에서 FIR 필터를 고려하여 시작하겠습니다. 

In [3]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in * b0.U(8.W) + x_n1 * b1.U(8.W) +
    x_n2 * b2.U(8.W) + x_n3 * b3.U(8.W)
}

defined class My4ElementFir

이 회로는 다음을 사용하여 이 4탭 필터의 버전을 생성할 수 있기 때문에 생성기의 간단한 경우입니다.
다른 계수. 그러나 회로에 더 많은 탭이 있기를 원한다면 어떻게 될까요? 우리는 이것을 여러 단계로 할 것입니다.

- 탭 구성 가능한 FIR의 소프트웨어 *골든 모델*을 구축합니다.
- 이 모델을 사용하도록 테스트를 다시 디자인하고 작동하는지 확인합니다.
- 구성 가능한 탭 수를 허용하도록 My4ElementFir를 리팩터링합니다.
- 새로운 테스트 하니스를 사용하여 새로운 회로를 테스트합니다.

<span style="color:blue">**Example: FIR Golden Model**</span><br><a name="fir-golden-model"></a> 
아래는 FIR 회로의 Scala 소프트웨어 구현입니다.

In [5]:
/**
  * A naive implementation of an FIR filter with an arbitrary number of taps.
  */
class ScalaFirFilter(taps: Seq[Int]) { //Seq
  var pseudoRegisters = List.fill(taps.length)(0) //Registers

  def poke(value: Int): Int = { //Poke
    pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1) //Updating the registers
    var accumulator = 0
    for(i <- taps.indices) {
      accumulator += taps(i) * pseudoRegisters(i) //Computing the output
    }
    accumulator
  }
}

defined class ScalaFirFilter

### Seq
`taps`가 `Seq[Int]`가 되었음을 유의하십시오. 이는 클래스 사용자가 클래스를 구성할 때 임의의 긴 `Int` 시퀀스를 전달할 수 있음을 의미합니다.

### Registers
`var pseudoRegisters = List.fill(taps.length)(0)`을 사용하여 이전 주기의 값을 보유할 `List`를 만듭니다. `List`를 선택한 이유는 헤드에 요소를 추가하고 마지막 요소를 제거하는 구문이 매우 간단하기 때문입니다. scala 컬렉션 패밀리의 거의 모든 구성원을 사용할 수 있습니다. 우리는 또한 모든 0을 포함하도록 이 목록을 초기화하고 있습니다.

### Poke
우리 클래스는 필터에 새로운 입력을 넣고 Clock을 순환하는 것을 에뮬레이트하는 poke 함수/메서드를 추가합니다.

### Updating the registers
`pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)` 행은 먼저 List의 `take` 메소드를 사용하여 List의 마지막 요소를 제외한 모든 요소를 유지한 다음 `::` List 연결 연산자를 사용합니다. 축소된 List의 head 부분에 `value`를 추가합니다.

### Computing the output
누산기가 있는 간단한 for 루프는 목록의 각 요소에 해당 탭 계수를 곱한 값을 합산합니다. `accumulator`만 있는 줄은 해당 값을 함수 결과로 반환합니다.

## Adapting our previous test for testing our golden model
이제 이전 작업을 사용하여 golden model이 작동하는지 확인합니다. 약간의 편집 마법이 우리의 이전 테스트를 활용하여 다음과 같이 모핑합니다.

In [ ]:
val filter = new ScalaFirFilter(Seq(1, 1, 1, 1))

var out = 0

out = filter.poke(1)
println(s"out = $out")
assert(out == 1)  // 1, 0, 0, 0

out = filter.poke(4)
assert(out == 5)  // 4, 1, 0, 0
println(s"out = $out")

out = filter.poke(3)
assert(out == 8)  // 3, 4, 1, 0
println(s"out = $out")

out = filter.poke(2)
assert(out == 10)  // 2, 3, 4, 1
println(s"out = $out")

out = filter.poke(7)
assert(out == 16)  // 7, 2, 3, 4
println(s"out = $out")

out = filter.poke(0)
assert(out == 12)  // 0, 7, 2, 3
println(s"out = $out")

이전 블록을 실행하면 소프트웨어 모델이 My4ElementFir와 동일한 결과를 반환함을 보여줍니다.


## Test circuit using the golden model.<a name="use-golden-model-as-test"></a> 
이제 골든 모델에 대해 합리적으로 확신을 얻었으므로 힘들게 손으로 만든 예제를 사용하는 대신 회로 출력을 골든 모델의 출력과 비교하기 위해 테스트를 다시 작성합니다.
다음은 이를 수행하기 위한 빠른 첫 번째 패스입니다.

In [ ]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

test(new My4ElementFir(1, 1, 1, 1)) { c =>
    for(i <- 0 until 100) {
        val input = scala.util.Random.nextInt(8)

        val goldenModelResult = goldenModel.poke(input)

        c.io.in.poke(input.U)

        c.io.out.expect(goldenModelResult.U, s"i $i, input $input, gm $goldenModelResult, ${c.io.out.peek().litValue}")

        c.clock.step(1)
    }

}

우리의 테스트는 100주기 동안 실행되며 하드웨어와 소프트웨어라는 두 가지 다른 방법이 각 단계에서 동기화되어 있는지 확인합니다.

### Things to watch out for
(즉, 이 글을 작성하는 동안 실제로 저지른 실수입니다.)

1. 올바른 위치에서 단계를 가져옵니다. 소프트웨어와 하드웨어는 다르게 실행됩니다. 이것을 잘못 이해하기 쉽습니다.
1. 이 테스트는 IO와 레지스터의 크기에 매우 민감하기 때문에 취약합니다. 임의의 데이터 비트 폭에서 래핑 동작을 관찰하는 소프트웨어 골든 모델을 구현하는 것은 복잡할 수 있습니다. 여기서 우리는 맞는 값만 전달하는지 확인합니다.

<span style="color:blue">**Example: Parameterized FIR Generator**</span><br><a name="fir-golden-model"></a> 
아래에서 탭에 사용할 상수의 `Seq`를 사용하는 새 필터 클래스 `MyManyElementsFilter`를 만들었습니다. 이 목록은 요소 수에 제한이 없습니다.
좋은 측정을 위해 회로에서 처리할 수 있는 숫자의 크기를 제어할 수 있는 `bitWidth`가 추가되었습니다.
가변 길이에 대한 응답으로 레지스터 생성 및 연결 방법을 리팩토링해야 했습니다.
아래에 사용된 방법론은 사용 가능한 컬렉션 함수 라이브러리의 간단한 하위 집합을 사용합니다.
이후 섹션에서는 상황을 보다 명확하게 만드는 방식으로 동작을 보다 간결하게 표현하는 방법을 보여줍니다.

In [ ]:
class MyManyElementFir(consts: Seq[Int], bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val regs = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) regs += io.in
      else       regs += RegNext(regs(i - 1), 0.U)
  }
  
  val muls = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      muls += regs(i) * consts(i).U
  }

  val scan = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) scan += muls(i)
      else scan += muls(i) + scan(i - 1)
  }

  io.out := scan.last
}

#### How we did it
라인 7, 13, 18에서 시작하는 세 개의 평행 섹션이 있습니다.
우리는 'ArrayBuffer'라는 Scala 컬렉션 유형을 사용하고 있습니다.
`ArrayBuffer`를 사용하면 `+=` 연산자를 사용하여 요소를 추가할 수 있습니다(삽입 및 삭제도 가능하지만 필요하지 않음).
먼저 요소가 `UInt`인 ArrayBuffer `regs`를 만듭니다.
그런 다음 탭을 반복하여 입력을 첫 번째 요소로 추가한 다음 RegNext를 사용하여 레지스터를 생성하여 레지스터의 입력을 이전 요소(`regs(i-1)`)에 연결하고 이를 부호 없는 0으로 초기화합니다(`0. 유`).
이 레지스터는 필요할 때 입력의 이전 값을 유지합니다.

다음으로, `UInt`의 또 다른 ArrayBuffer `muls`를 생성합니다.
muls의 각 요소는 i번째 요소가 `regs(i)`와 `const(i)`의 곱인 노드가 됩니다.

'scan.last' 메서드의 사용에 유의하십시오.
컬렉션의 마지막 요소를 취하며 `regs` 생성 중에 사용되는 `regs(i - 1)`에 대한 보다 우아한 대안입니다.

### Does it behave the same as `My4ElementFir`?
새 버전의 좋은 첫 번째 테스트는 방금 적용한 테스트를 통과할 수 있는지 확인하는 것입니다.
'My4ElementFir'.
우리는 `MyManyElementFir`의 인스턴스를 만들고 이를 통해 더 많은 데이터를 실행합니다.

In [ ]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

test(new MyManyElementFir(Seq(1, 1, 1, 1), 8)) { c =>
    for(i <- 0 until 100) {
      val input = scala.util.Random.nextInt(8)

      val goldenModelResult = goldenModel.poke(input)

      c.io.in.poke(input.U)

      c.io.out.expect(goldenModelResult.U, s"i $i, input $input, gm $goldenModelResult, ${c.io.out.peek().litValue}")

      c.clock.step(1)
    }
}

### Now let's test a bunch of different sized FIR filters
우리는 몇 가지 도우미 함수를 만듭니다: 난수를 가져오는 `r`; 'runOneTest'는 특정 탭 세트에 대한 필터의 하드웨어 시뮬레이션 및 골든 모델을 생성한 다음 필터를 통해 데이터 탭 수의 최소 두 배를 실행합니다.

In [ ]:
/** a convenience method to get a random integer
  */
def r(): Int = {
  scala.util.Random.nextInt(1024)
}

/**
  * run a test comparing software and hardware filters
  * run for at least twice as many samples as taps
  */
def runOneTest(taps: Seq[Int]) {
    val goldenModel = new ScalaFirFilter(taps)

    test(new MyManyElementFir(taps, 32)) { c =>
        for(i <- 0 until 2 * taps.length) {
            val input = r()

            val goldenModelResult = goldenModel.poke(input)

            c.io.in.poke(input.U)

            c.io.out.expect(goldenModelResult.U, s"i $i, input $input, gm $goldenModelResult, ${c.io.out.peek().litValue}")

            c.clock.step(1)
        }
    }
}

for(tapSize <- 2 until 100 by 10) {
    val taps = Seq.fill(tapSize)(r())  // create a sequence of random coefficients

    runOneTest(taps)
}

### Just for fun, let's make a bigger one
다음은 500 탭에서 단일 테스트를 실행합니다.
FIR 필터. 실행하는 데 1분 정도 걸릴 수 있습니다.
(힌트: 실행이 완료되면 도구 모음에서 Scala가 ● Scala ○로 변경되는지 확인하십시오.)

In [ ]:
runOneTest(Seq.fill(500)(r()))

---
# Hardware Collections

<span style="color:blue">**Example: Add run-time configurable taps to our FIR**</span><br>
다음 코드는 FIR 생성기의 IO에 `consts` 벡터를 추가하여 회로 생성 후 외부에서 coefficient를 변경할 수 있도록 합니다.
이것은 Chisel 컬렉션 유형 `Vec`으로 수행됩니다.
`Vec`은 많은 스칼라 collection method를 지원하지만 Chisel 하드웨어 요소만 포함할 수 있습니다.
`Vec`은 일반 Scala 컬렉션이 작동하지 않는 상황에서만 사용해야 합니다.
기본적으로 이것은 두 가지 상황 중 하나입니다.
1. 번들(일반적으로 IO로 사용되는 번들)에 요소 모음이 필요합니다.
1. 하드웨어의 일부인 인덱스를 통해 컬렉션에 액세스해야 합니다(Register File을 생각해보세요).


In [ ]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })

  // Reference solution
  val regs = RegInit(VecInit(Seq.fill(length - 1)(0.U(8.W))))
  for(i <- 0 until length - 1) {
      if(i == 0) regs(i) := io.in
      else       regs(i) := regs(i - 1)
  }
  
  val muls = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) muls(i) := io.in * io.consts(i)
      else       muls(i) := regs(i - 1) * io.consts(i)
  }

  val scan = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) scan(i) := muls(i)
      else scan(i) := muls(i) + scan(i - 1)
  }

  io.out := scan(length - 1)
}

In [ ]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

test(new MyManyDynamicElementVecFir(4)) { c =>
    c.io.consts(0).poke(1.U)
    c.io.consts(1).poke(1.U)
    c.io.consts(2).poke(1.U)
    c.io.consts(3).poke(1.U)
    for(i <- 0 until 100) {
        val input = scala.util.Random.nextInt(8)

        val goldenModelResult = goldenModel.poke(input)

        c.io.in.poke(input.U)

        c.io.out.expect(goldenModelResult.U, s"i $i, input $input, gm $goldenModelResult, ${c.io.out.peek().litValue}")

        c.clock.step(1)
    }
}


<span style="color:red">**Exercise: 32-bit RISC-V Processor**</span><br><a name="fir-golden-model"></a>

[register file](https://en.wikipedia.org/wiki/Register_file)은 프로세서를 만들기 위한 중요한 빌딩 블록입니다.
레지스터 파일은 여러 읽기 또는 쓰기 포트를 통해 읽거나 쓸 수 있는 레지스터 배열입니다.
각 포트는 주소와 데이터 필드로 구성됩니다.

[RISC-V 명령어 세트 아키텍처](https://riscv.org/specifications/)는 여러 변형을 정의하며 그 중 가장 간단한 것이 RV32I입니다.
RV32I에는 32비트 레지스터의 크기 32 어레이가 있습니다.
**인덱스 0의 레지스터(첫 번째 레지스터)는 무엇을 기록하든 상관없이 읽을 때 항상 0입니다**(0이 있으면 편리합니다).

단일 쓰기 포트와 매개변수화된 읽기 포트 수를 사용하여 RV32I용 레지스터 파일을 구현합니다.
쓰기는 `wen`(쓰기 활성화)이 어설션된 경우에만 수행됩니다.

In [9]:
class RegisterFile(readPorts: Int) extends Module {
    require(readPorts >= 0)
    val io = IO(new Bundle {
        val wen   = Input(Bool())
        val waddr = Input(UInt(5.W))
        val wdata = Input(UInt(32.W))
        val raddr = Input(Vec(readPorts, UInt(5.W)))
        val rdata = Output(Vec(readPorts, UInt(32.W)))
    })
    
    // A Register of a vector of UInts
    val reg = RegInit(VecInit(Seq.fill(32)(0.U(32.W))))
    
    ???
    
    
}

defined class RegisterFile

In [7]:
test(new RegisterFile(2) ) { c =>
  def readExpect(addr: Int, value: Int, port: Int = 0): Unit = {
    c.io.raddr(port).poke(addr.U)
    c.io.rdata(port).expect(value.U)
  }
  def write(addr: Int, value: Int): Unit = {
    c.io.wen.poke(true.B)
    c.io.wdata.poke(value.U)
    c.io.waddr.poke(addr.U)
    c.clock.step(1)
    c.io.wen.poke(false.B)
  }
  // everything should be 0 on init
  for (i <- 0 until 32) {
    readExpect(i, 0, port = 0)
    readExpect(i, 0, port = 1)
  }

  // write 5 * addr + 3
  for (i <- 0 until 32) {
    write(i, 5 * i + 3)
  }

  // check that the writes worked
  for (i <- 0 until 32) {
    readExpect(i, if (i == 0) 0 else 5 * i + 3, port = i % 2)
  }
}

Elaborating design...
Done elaborating.
test RegisterFile Success: 0 tests passed in 34 cycles in 0.043846 seconds 775.44 Hz


<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
    when (io.wen) {
        reg(io.waddr) := io.wdata
    }
    for (i &lt;- 0 until readPorts) {
        when (io.raddr(i) === 0.U) {
            io.rdata(i) := 0.U
        } .otherwise {
            io.rdata(i) := reg(io.raddr(i))
        }
    }

</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)